<a href="https://colab.research.google.com/github/EEarth1270/pizzaTfClassify/blob/master/AiForReBuild.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import tensorflow as tf
import PIL
import pathlib
import sys
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, classification, accuracy_score
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import numpy as np

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_dir = os.path.join('/content/drive/My Drive/model/pictureforbuildmodel','*','*')
dataset_path = glob(dataset_dir)
df = pd.DataFrame(dict(path=dataset_path))

In [ ]:
source_enc = LabelEncoder()
df['is_pizza'] = df['path'].map(lambda x: x.split('/')[-2].replace('_', ' ').strip())
df['source_id'] = source_enc.fit_transform(df['is_pizza'])

In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   path       2944 non-null   object
 1   is_pizza   2944 non-null   object
 2   source_id  2944 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 69.1+ KB
None


In [ ]:
Img_size=(256,256)
train_df, test_df = train_test_split(df, test_size=0.3,
                                     random_state=1,
                                     stratify=df[['source_id']])
train_dfe, val_df = train_test_split(train_df, random_state=1, test_size=0.2, stratify=train_df[['source_id']])
TrainImageDataGenerator = ImageDataGenerator(
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        height_shift_range=0.1,
                                        width_shift_range=0.1,
                                        rotation_range=5,
                                        shear_range=0.01,
                                        fill_mode='reflect',
                                        zoom_range=0.15,
                                        rescale=1. / 255.
                                       )
testImageGenerator = ImageDataGenerator(rescale=1. / 255.)

train_gen = TrainImageDataGenerator.flow_from_dataframe(
    train_dfe, x_col='path', y_col='is_pizza', class_mode='binary', batch_size=28,
    target_size=Img_size
)
val_gen = TrainImageDataGenerator.flow_from_dataframe(
    val_df, x_col='path', y_col='is_pizza', class_mode='binary', batch_size=28,
    target_size=Img_size
)
test_gen = testImageGenerator.flow_from_dataframe(
    test_df, x_col='path', y_col='is_pizza', class_mode='binary', batch_size=28,shuffle=False,
    target_size=Img_size,
)

Found 1648 validated image filenames belonging to 2 classes.
Found 412 validated image filenames belonging to 2 classes.
Found 884 validated image filenames belonging to 2 classes.


In [ ]:
(X, y) = test_gen.next()
print(X.shape, X.mean(), y)

print(test_df.head())

(28, 256, 256, 3) 0.4886008 [0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0.
 0. 1. 1. 0.]
                                                   path   is_pizza  source_id
2803  /content/drive/My Drive/model/pictureforbuildm...  not pizza          0
2940  /content/drive/My Drive/model/pictureforbuildm...  not pizza          0
1304  /content/drive/My Drive/model/pictureforbuildm...      pizza          1
746   /content/drive/My Drive/model/pictureforbuildm...      pizza          1
26    /content/drive/My Drive/model/pictureforbuildm...      pizza          1


In [ ]:
model = Sequential([
    # layers.experimental.preprocessing.Rescaling(scale=1. / 255, input_shape=(300, 300, 3)),
    # layers.LayerNormalization(input_shape=(300, 300, 3)),
    layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(256, 256, 3)),
    layers.MaxPool2D(pool_size=(2, 2)),

    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.MaxPool2D(pool_size=(2, 2)),

    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.MaxPool2D(pool_size=(2, 2)),

    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.MaxPool2D(pool_size=(2, 2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid'),

])

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 64)        3

In [ ]:
## load model 
model = load_model('/content/drive/My Drive/model/modeloncolabv1epoch30')

In [ ]:
epochs = 50

In [ ]:
history = model.fit(train_gen,
                    validation_data=val_gen,
                    epochs=epochs,
                    )
model.save('/content/drive/My Drive/model/modeloncolabv1epoch50')

Epoch 1/50
59/59 [==============================] - 1046s 18s/step - loss: 0.6428 - accuracy: 0.6183 - val_loss: 0.6533 - val_accuracy: 0.6432
Epoch 2/50
59/59 [==============================] - 51s 862ms/step - loss: 0.4301 - accuracy: 0.8107 - val_loss: 0.3906 - val_accuracy: 0.8083
Epoch 3/50
59/59 [==============================] - 50s 856ms/step - loss: 0.3283 - accuracy: 0.8647 - val_loss: 0.3160 - val_accuracy: 0.8689
Epoch 4/50
59/59 [==============================] - 50s 852ms/step - loss: 0.2685 - accuracy: 0.8950 - val_loss: 0.2866 - val_accuracy: 0.8883
Epoch 5/50
59/59 [==============================] - 50s 856ms/step - loss: 0.2569 - accuracy: 0.8981 - val_loss: 0.2769 - val_accuracy: 0.8811
Epoch 6/50
59/59 [==============================] - 50s 855ms/step - loss: 0.2554 - accuracy: 0.8944 - val_loss: 0.2559 - val_accuracy: 0.9053
Epoch 7/50
59/59 [==============================] - 50s 848ms/step - loss: 0.2613 - accuracy: 0.8932 - val_loss: 0.4028 - val_accuracy: 0.8277

In [ ]:
def predict_pizza(x):
    per = x[0]*100
    if per >= 60.0:
        return 'pizza'
    else:
        return 'not_pizza'


In [ ]:
predictions = model.predict(test_gen)

In [ ]:
print(predictions)

[[1.00585516e-03]
 [7.00684439e-04]
 [9.99552906e-01]
 [9.79498327e-01]
 [9.99917507e-01]
 [9.99168634e-01]
 [9.99912500e-01]
 [9.99739468e-01]
 [9.96980846e-01]
 [9.97749150e-01]
 [3.27413768e-01]
 [5.37088476e-02]
 [9.96792495e-01]
 [9.99997497e-01]
 [5.31443278e-04]
 [2.60354765e-02]
 [4.57420974e-04]
 [9.41914976e-01]
 [9.99487758e-01]
 [8.79698992e-01]
 [9.98209000e-01]
 [4.74048342e-04]
 [5.23093104e-01]
 [3.21481680e-03]
 [6.70315567e-05]
 [9.94251132e-01]
 [9.99939799e-01]
 [2.62893766e-01]
 [9.99875426e-01]
 [9.99712646e-01]
 [9.76042628e-01]
 [1.39574334e-01]
 [2.36227969e-03]
 [3.32490774e-03]
 [2.11143936e-03]
 [9.89651501e-01]
 [9.06217396e-01]
 [9.99872684e-01]
 [4.41228694e-05]
 [7.05999613e-01]
 [1.66819063e-05]
 [4.77651320e-02]
 [5.02935378e-04]
 [2.07906328e-02]
 [9.99729693e-01]
 [2.26259988e-04]
 [9.97496307e-01]
 [9.99590337e-01]
 [1.00000000e+00]
 [1.16260699e-03]
 [9.99961019e-01]
 [1.21517833e-02]
 [2.12243642e-04]
 [4.10329318e-04]
 [9.99992609e-01]
 [1.583908

In [ ]:
print(test_df.info())
len(predictions)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 884 entries, 2803 to 1933
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   path       884 non-null    object
 1   is_pizza   884 non-null    object
 2   source_id  884 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 27.6+ KB
None


884

In [ ]:
test_df.loc[2803,'pred_pizz']= 'pizza'
count = 0
for x in predictions:
  a = predict_pizza(x)
  test_df.iloc[count,3] = a
  print(a)
  print(count)
  count=count+1



not_pizza
0
not_pizza
1
pizza
2
pizza
3
pizza
4
pizza
5
pizza
6
pizza
7
pizza
8
pizza
9
not_pizza
10
not_pizza
11
pizza
12
pizza
13
not_pizza
14
not_pizza
15
not_pizza
16
pizza
17
pizza
18
pizza
19
pizza
20
not_pizza
21
not_pizza
22
not_pizza
23
not_pizza
24
pizza
25
pizza
26
not_pizza
27
pizza
28
pizza
29
pizza
30
not_pizza
31
not_pizza
32
not_pizza
33
not_pizza
34
pizza
35
pizza
36
pizza
37
not_pizza
38
pizza
39
not_pizza
40
not_pizza
41
not_pizza
42
not_pizza
43
pizza
44
not_pizza
45
pizza
46
pizza
47
pizza
48
not_pizza
49
pizza
50
not_pizza
51
not_pizza
52
not_pizza
53
pizza
54
not_pizza
55
pizza
56
pizza
57
not_pizza
58
pizza
59
not_pizza
60
not_pizza
61
pizza
62
pizza
63
not_pizza
64
not_pizza
65
not_pizza
66
pizza
67
not_pizza
68
pizza
69
not_pizza
70
pizza
71
not_pizza
72
not_pizza
73
not_pizza
74
pizza
75
pizza
76
not_pizza
77
pizza
78
pizza
79
not_pizza
80
not_pizza
81
not_pizza
82
not_pizza
83
pizza
84
pizza
85
pizza
86
pizza
87
not_pizza
88
pizza
89
not_pizza
90
pizza
91
no

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


160
not_pizza
161
pizza
162
pizza
163
not_pizza
164
not_pizza
165
pizza
166
pizza
167
not_pizza
168
not_pizza
169
pizza
170
not_pizza
171
not_pizza
172
pizza
173
pizza
174
pizza
175
not_pizza
176
not_pizza
177
not_pizza
178
not_pizza
179
pizza
180
not_pizza
181
not_pizza
182
pizza
183
pizza
184
pizza
185
not_pizza
186
pizza
187
not_pizza
188
pizza
189
not_pizza
190
pizza
191
pizza
192
pizza
193
pizza
194
not_pizza
195
pizza
196
pizza
197
not_pizza
198
pizza
199
not_pizza
200
not_pizza
201
not_pizza
202
pizza
203
pizza
204
not_pizza
205
pizza
206
not_pizza
207
not_pizza
208
not_pizza
209
not_pizza
210
pizza
211
not_pizza
212
not_pizza
213
pizza
214
not_pizza
215
not_pizza
216
not_pizza
217
not_pizza
218
not_pizza
219
pizza
220
not_pizza
221
not_pizza
222
not_pizza
223
not_pizza
224
pizza
225
not_pizza
226
pizza
227
not_pizza
228
not_pizza
229
not_pizza
230
not_pizza
231
pizza
232
not_pizza
233
pizza
234
not_pizza
235
pizza
236
pizza
237
pizza
238
pizza
239
pizza
240
pizza
241
not_pizza


In [ ]:
print(test_df.head())

                                                   path  ...  pred_pizz
2803  /content/drive/My Drive/model/pictureforbuildm...  ...  not_pizza
2940  /content/drive/My Drive/model/pictureforbuildm...  ...  not_pizza
1304  /content/drive/My Drive/model/pictureforbuildm...  ...      pizza
746   /content/drive/My Drive/model/pictureforbuildm...  ...      pizza
26    /content/drive/My Drive/model/pictureforbuildm...  ...      pizza

[5 rows x 4 columns]


In [ ]:
from sklearn.metrics import confusion_matrix
a = confusion_matrix(test_df.is_pizza,test_df.pred_pizz)
print(a)

[[  0 416  26]
 [  0   0   0]
 [  0  23 419]]
